# DART 오픈 API 01 - 개요와 활용

- DART 오픈AP 사용법
- json_normalize() 활용 
- 기업개황 API
- 정보검색 API
- 개별 보고서와 하위 보고서 URL 얻기
- 첨부파일과 재무제표 엑셀 파일 가져오기

### 오픈 API

- 공시 보고서 목록를 실시간 제공
- https://dart.fss.or.kr/dsap001/intro.do 오픈API 소개
- https://dart.fss.or.kr/dsap001/guide.do API 개발 가이드

### API 인증키 발급

- 계정을 만들고, API 인증키 발급 받는다

- 계정 신청: https://dart.fss.or.kr/dsag002/insertForm.do

### 개인용, 기업용
- 개인용: 1만건/일
- 기업용: 조회 건수 제한이 없는 대신 조회 서버 IP 제한

### 인증키
- 알파벳과 숫자로 이루어진 40 바이트 문자열
- auth 파라미터로 제공

http://dart.fss.or.kr/api/search.xml?auth=e81e78aa519d1a1c4ec7867ef12a937ab6cbd4c7

### 오픈 API가 제공하는 주요정보

 - 주요하게 다음 2가지 API를 제공한다
    1. 기업 개황 API : http://dart.fss.or.kr/api/company.json?auth=xxx&crp_cd=xxx
    2. 정보 검색 API : http://dart.fss.or.kr/api/search.json?auth=xxx



## API 개발 가이드 사이트에서 테스트

- https://dart.fss.or.kr/dsap001/guide.do API 개발 가이드

![](http://i.imgur.com/9NpoJgx.png)

In [2]:
## 현재 발급받은 API인증키(2018-06-03)
auth = '9daed961201d085d01dfb40c8a1acb5c3fd41b85'

# 헤더: 실제 인증키를 사용시에는 사용하지 않아도 된다
headers={'Referer':'https://dart.fss.or.kr/dsap001/guide.do'}

In [3]:
import requests

url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}'
url = url_tmpl.format(auth=auth)

r = requests.get(url, headers=headers)
r.text

'{"err_code":"000","err_msg":"정상","page_no":1,"page_set":10,"total_count":158,"total_page":16,"list":[{"crp_cls":"K","crp_nm":"크레아플래닛","crp_cd":"058530","rpt_nm":"상호변경안내","rcp_no":"20180605900214","flr_nm":"크레아플래닛","rcp_dt":"20180605","rmk":"코"},{"crp_cls":"Y","crp_nm":"미래에셋대우","crp_cd":"006800","rpt_nm":"투자설명서(일괄신고)","rcp_no":"20180605000156","flr_nm":"미래에셋대우","rcp_dt":"20180605","rmk":""},{"crp_cls":"Y","crp_nm":"미래에셋대우","crp_cd":"006800","rpt_nm":"투자설명서(일괄신고)","rcp_no":"20180605000154","flr_nm":"미래에셋대우","rcp_dt":"20180605","rmk":""},{"crp_cls":"Y","crp_nm":"삼성전자","crp_cd":"005930","rpt_nm":"임원ㆍ주요주주특정증권등소유상황보고서","rcp_no":"20180605000153","flr_nm":"김희덕","rcp_dt":"20180605","rmk":""},{"crp_cls":"K","crp_nm":"크레아플래닛","crp_cd":"058530","rpt_nm":"임시주주총회결과","rcp_no":"20180605900209","flr_nm":"크레아플래닛","rcp_dt":"20180605","rmk":"코"},{"crp_cls":"Y","crp_nm":"엔에스쇼핑","crp_cd":"138250","rpt_nm":"임원ㆍ주요주주특정증권등소유상황보고서","rcp_no":"20180605000150","flr_nm":"한국투자밸류자산운용","rcp_dt":"20180605","rmk":""},{"

In [4]:
from pandas.io.json import json_normalize
import json
import pandas as pd


json_str = '''
[
    {"A":1, "B":2, "C":3},
    {"A":4, "B":5, "C":6},
    {"A":7, "B":8, "C":9}
]
'''

jo = json.loads(json_str)
result = json_normalize(jo)
result

,A,B,C
0,1,2,3
1,4,5,6
2,7,8,9


In [5]:
url = 'http://dart.fss.or.kr/api/company.json?auth={auth}&crp_cd={code}'
url = url.format(auth=auth, code ='005930')
r = requests.get(url)
jo = json.loads(r.text)
jo

{'acc_mt': '12',
 'adr': '경기도 수원시 영통구  삼성로 129 (매탄동)',
 'bsn_no': '1248100998',
 'ceo_nm': '김기남, 김현석, 고동진',
 'crp_cls': 'Y',
 'crp_nm': '삼성전자(주)',
 'crp_nm_e': 'SAMSUNG ELECTRONICS CO,.LTD',
 'crp_nm_i': '삼성전자',
 'crp_no': '1301110006246',
 'err_code': '000',
 'err_msg': '정상',
 'est_dt': '19690113',
 'fax_no': '031-200-7538',
 'hm_url': 'www.sec.co.kr',
 'ind_cd': '264',
 'ir_url': '',
 'phn_no': '031-200-1114',
 'stock_cd': '005930'}

### 위 결과물의 내용
- err_code: 에러코드(오류 메시지 참조)
- err_msg: 에러메시지(오류 메시지 참조)
- crp_nm: 정식명칭
- crp_nm_e: 영문명칭
- crp_nm_i: 종목명(상장사) 또는 약식명칭(기타법인)
- stock_cd: 상장회사인 경우 주식의 종목코드
- ceo_nm: 대표자명
- crp_cls: 법인구분 : Y(유가), K(코스닥), N(코넥스), E(기타)
- crp_no: 법인등록번호
- bsn_no: 사업자등록번호
- adr: 주소
- hm_url: 홈페이지
- ir_url: IR홈페이지
- phn_no: 전화번호
- fax_no: 팩스번호
- ind_cd: 업종코드
- est_dt: 설립일(YYYYMMDD)
- acc_mt: 결산월(MM)


In [6]:
jo['bsn_no'] #사업자 등록번호

'1248100998'

### 정보 검색 API

- http://dart.fss.or.kr/api/search.json?auth={auth}&crp_cd={code}&start_dt=19990101&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003 
    - (회사의 전체 사업+반기+분기보고서)
- crp_cd: 공시대상회사의 종목코드(상장사:숫자 6자리) 또는 고유번호(기타법인:숫자 8자리)
- end_dt: 검색종료 접수일자(YYYYMMDD) : 없으면 당일
- start_dt: 검색시작 접수일자(YYYYMMDD) : 없으면 end_dt crp_cd가 없는 경우 검색기간은 3개월로 제한
- fin_rpt: 최종보고서만 검색여부(Y or N) 기본값 : N (정정이 있는 경우 최종정정만 검색)
- dsp_tp: 정기공시(A), 주요사항보고(B), 발행공시(C), 지분공시(D), 기타공시(E), 외부감사관련(F), 펀드공시(G), 자산유동화(H), 거래소공시(I), 공정위공시(J)
- bsn_tp: 정기공시(5), 주요사항보고(3), 발행공시(11), 지분공시(4), 기타공시(9), 외부감사관련(3), 펀드공시(3), 자산유동화(6), 거래소공시(6), 공정위공시(5) 
- sort: 접수일자(date), 회사명(crp), 보고서명(rpt) 기본값 : date
- series: 오름차순(asc), 내림차순(desc) 기본값 : desc
- page_no: 페이지 번호(1~n) 기본값 : 1
- page_set: 페이지당 건수(1~100) 기본값 : 10, 최대값 : 100
- callback: 콜백함수명(JSONP용)


In [7]:
url = 'http://dart.fss.or.kr/api/search.json?auth={auth}&crp_cd={code}&start_dt=19990101&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003' 
url = url.format(auth=auth, code ='005930')
r = requests.get(url)
jo = json.loads(r.text)
jo

{'err_code': '000',
 'err_msg': '정상',
 'list': [{'crp_cd': '005930',
   'crp_cls': 'Y',
   'crp_nm': '삼성전자',
   'flr_nm': '삼성전자',
   'rcp_dt': '20180515',
   'rcp_no': '20180515001699',
   'rmk': '',
   'rpt_nm': '분기보고서 (2018.03)'},
  {'crp_cd': '005930',
   'crp_cls': 'Y',
   'crp_nm': '삼성전자',
   'flr_nm': '삼성전자',
   'rcp_dt': '20180402',
   'rcp_no': '20180402005019',
   'rmk': '연',
   'rpt_nm': '사업보고서 (2017.12)'},
  {'crp_cd': '005930',
   'crp_cls': 'Y',
   'crp_nm': '삼성전자',
   'flr_nm': '삼성전자',
   'rcp_dt': '20171114',
   'rcp_no': '20171114001694',
   'rmk': '',
   'rpt_nm': '분기보고서 (2017.09)'},
  {'crp_cd': '005930',
   'crp_cls': 'Y',
   'crp_nm': '삼성전자',
   'flr_nm': '삼성전자',
   'rcp_dt': '20170814',
   'rcp_no': '20170814001832',
   'rmk': '',
   'rpt_nm': '반기보고서 (2017.06)'},
  {'crp_cd': '005930',
   'crp_cls': 'Y',
   'crp_nm': '삼성전자',
   'flr_nm': '삼성전자',
   'rcp_dt': '20170515',
   'rcp_no': '20170515003806',
   'rmk': '',
   'rpt_nm': '분기보고서 (2017.03)'},
  {'crp_cd': '0059

In [28]:
# 당일접수 10건보기(주말 공휴일에는 접수가 없을 수 있음...)

url = 'http://dart.fss.or.kr/api/search.json?auth={auth}&page_set=10'.format(auth=auth)
r = requests.get(url, headers=headers)
jo = json.loads(r.text)
result = json_normalize(jo,'list')
result


,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,005930,Y,삼성전자,김대원,20180605,20180605000169,,임원ㆍ주요주주특정증권등소유상황보고서
1,039200,K,오스코텍,지케이에셋,20180605,20180605000168,,주식등의대량보유상황보고서(약식)
2,058530,K,크레아플래닛,크레아플래닛,20180605,20180605000167,,주식매수선택권부여에관한신고
3,00164876,E,케이비증권,케이비증권,20180605,20180605000165,,증권발행실적보고서
4,01258127,E,에이블걸포,삼일회계법인,20180605,20180605000164,,감사보고서 (2018.02)
5,00684273,E,교세라한국,삼일회계법인,20180605,20180605000163,,감사보고서 (2018.03)
6,005930,Y,삼성전자,최주호,20180605,20180605000162,,임원ㆍ주요주주특정증권등소유상황보고서
7,016360,Y,삼성증권,삼성증권,20180605,20180605000161,,투자설명서(일괄신고)
8,090430,Y,아모레퍼시픽,강일권,20180605,20180605000160,,임원ㆍ주요주주특정증권등소유상황보고서
9,008560,Y,메리츠종금증권,메리츠종금증권,20180605,20180605000159,,투자설명서(일괄신고)


## 위 내용의 응답결과 내용
#### result
- err_code: 에러코드(오류 메시지 참조)
- err_msg: 에러메시지(오류 메시지 참조)
- page_no: 페이지 번호
- page_set: 페이지당 건수
- total_count: 총 건수
- total_page: 총 페이지 수

#### list
- crp_cls: 법인구분 : Y(유가), K(코스닥), N(코넥스), E(기타)
- crp_nm: 공시대상회사의 종목명(상장사) 또는 법인명(기타법인)
- crp_cd: 공시대상회사의 종목코드(6자리) 또는 고유번호(8자리)
- rpt_nm: 공시구분+보고서명+기타정보
    - [기재정정] : 본 보고서명으로 이미 제출된 보고서의 기재내용이 변경되어 제출된 것임
    - [첨부정정] : 본 보고서명으로 이미 제출된 보고서의 첨부내용이 변경되어 제출된 것임
    - [첨부추가] : 본 보고서명으로 이미 제출된 보고서의 첨부서류가 추가되어 제출된 것임
    - [변경등록] : 본 보고서명으로 이미 제출된 보고서의 유동화계획이 변경되어 제출된 것임
    - [연장결정] : 본 보고서명으로 이미 제출된 보고서의 신탁계약이 연장되어 제출된 것임
    - [발행조건확정] : 본 보고서명으로 이미 제출된 보고서의 유가증권 발행조건이 확정되어 제출된 것임
    - [정정명령부과] : 본 보고서에 대하여 금융감독원이 정정명령을 부과한 것임
    - [정정제출요구] : 본 보고서에 대하여 금융감독원이 정정제출요구을 부과한 것임

- rcp_no: 접수번호(공시뷰어 연결에 이용)
    - PC용 : http://dart.fss.or.kr/dsaf001/main.do?rcpNo=접수번호
    - 모바일용 : http://m.dart.fss.or.kr/html_mdart/MD1007.html?rcpNo=접수번호

- flr_nm: 공시 제출인명
- rcp_dt: 공시 접수일자(YYYYMMDD)
- rmk: 조합된 문자로 각각은 아래와 같은 의미가 있음
    - 유 : 본 공시사항은 한국거래소 유가증권시장본부 소관임
    - 코 : 본 공시사항은 한국거래소 코스닥시장본부 소관임
    - 채 : 본 문서는 한국거래소 채권상장법인 공시사항임
    - 넥 : 본 문서는 한국거래소 코넥스시장 소관임
    - 공 : 본 공시사항은 공정거래위원회 소관임
    - 연 : 본 보고서는 연결부분을 포함한 것임
    - 정 : 본 보고서 제출 후 정정신고가 있으니 관련 보고서를 참조하시기 바람
    - 철 : 본 보고서는 철회(간주)되었으니 관련 철회신고서(철회간주안내)를 참고하시기 바람 

In [32]:
# 당일 접수 100건 (최대 100건)

url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}&page_set=100'
url = url_tmpl.format(auth=auth)
r = requests.get(url, headers=headers)
jo = json.loads(r.text)
result = json_normalize(jo, 'list')
result

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,01164156,E,리시안셔스,효림회계법인,20180605,20180605000184,,감사보고서 (2018.02)
1,01188341,E,골드스푼제이와이제사차,이촌회계법인,20180605,20180605000181,,감사보고서 (2018.01)
2,01114540,E,더밴세곡제일차,효림회계법인,20180605,20180605000180,,감사보고서 (2018.02)
3,038340,K,UCI,머큐리어드바이저,20180605,20180605000179,,임원ㆍ주요주주특정증권등소유상황보고서
4,005940,Y,NH투자증권,NH투자증권,20180605,20180605000178,,투자설명서(일괄신고)
5,010170,K,대한광통신,대한광통신,20180605,20180605900235,코,증권발행결과(자율공시)
6,00977526,E,뉴스타미래제일차,효림회계법인,20180605,20180605000177,,감사보고서 (2018.02)
7,01235481,E,뉴스타구로,효림회계법인,20180605,20180605000176,,감사보고서 (2018.02)
8,00554352,E,아이큐어,아이큐어,20180605,20180605000175,,[기재정정]증권신고서(지분증권)
9,038340,K,UCI,머큐리어드바이저,20180605,20180605000174,,주식등의대량보유상황보고서(일반)


In [38]:
### ★★★★★★★★★ 특정공시 추출하기

for i in range(0,100):
    if result.loc[i]['rpt_nm'] == '단일판매ㆍ공급계약체결(자율공시)':
        print(result.loc[i])

crp_cd                053590
crp_cls                    K
crp_nm               한국테크놀로지
flr_nm               한국테크놀로지
rcp_dt              20180605
rcp_no        20180605900123
rmk                        코
rpt_nm     단일판매ㆍ공급계약체결(자율공시)
Name: 91, dtype: object


In [49]:
# 회사의 전체 사업+반기+분기보고서 100건

url = 'http://dart.fss.or.kr/api/search.json?auth={auth}&crp_cd={code}&start_dt=19990101&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003&page_set=100'

url = url.format(auth=auth, code = '053590')
r = requests.get(url)
jo = json.loads(r.text)
result = json_normalize(jo, 'list')
print('총', len(result),'건')
result.head()

총 83 건


,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,053590,K,한국테크놀로지,한국테크놀로지,20180514,20180514005578,,분기보고서 (2018.03)
1,053590,K,한국테크놀로지,한국테크놀로지,20180330,20180330002539,,사업보고서 (2017.12)
2,053590,K,한국테크놀로지,한국테크놀로지,20171110,20171110000498,,분기보고서 (2017.09)
3,053590,K,한국테크놀로지,한국테크놀로지,20170811,20170811001095,,반기보고서 (2017.06)
4,053590,K,한국테크놀로지,한국테크놀로지,20170524,20170524000337,,[기재정정]사업보고서 (2016.12)


In [50]:
#보고서의 URL

df = result[:10] #처음 10개만

for i, row in df.iterrows():
    url_tmpl = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}"
    url = url_tmpl.format(row['rcp_no'])
    print (i, url)

0 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180514005578
1 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180330002539
2 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20171110000498
3 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170811001095
4 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170524000337
5 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170512005716
6 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170327000935
7 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20161111000727
8 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160812001034
9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160513004204


In [55]:
import requests
import re

def get_report_doc_urls(rcp_no):
    doc_urls = []
    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
    r = requests.get(url)
    reg = re.compile('viewDoc\((.*)\);')
    params = []
    matches = reg.findall(r.text)
    for m in matches:
        params.append(m.replace("'","").replace(" ","").split(","))
        
    doc_url_tmpl = "http://dart.fss.or.kr/report/viewer.do?rcpNo=%s&dcmNo=%s&eleId=%s&offset=%s&length=%s&dtd=%s" 
    
    for p in params:
        if rcp_no == p[0]:
            doc_urls.append(doc_url_tmpl % tuple(p))
    return doc_urls        


In [59]:
#사업보고서 하위 문서 전체
doc_urls = get_report_doc_urls('20180605900123') #문서명을 기준으로
for url in doc_urls:
    print(url)

http://dart.fss.or.kr/report/viewer.do?rcpNo=20180605900123&dcmNo=6206370&eleId=0&offset=0&length=0&dtd=HTML
